In [1]:
import zipfile
import os
import pandas as pd
import numpy as np

In [2]:
zip_file_path = 'C:/Users/Dell/Documents/My_repo/goit-KATRUSHENKO/HW5/homework.zip'

extract_path = 'C:/Users/Dell/Documents/My_repo/goit-KATRUSHENKO/HW5/homework/data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:

def load_files_from_folder(folder_path):
    files = os.listdir(folder_path)
    data_list = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, header=0)
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna()
        data_list.append(df)

    return pd.concat(data_list, ignore_index=True)
data_folder = 'C:/Users/Dell/Documents/My_repo/goit-KATRUSHENKO/HW5/homework/data/data'

In [4]:
idle_data = load_files_from_folder(os.path.join(data_folder, 'idle'))
running_data = load_files_from_folder(os.path.join(data_folder, 'running'))
stairs_data = load_files_from_folder(os.path.join(data_folder, 'stairs'))
walking_data = load_files_from_folder(os.path.join(data_folder, 'walking'))

In [5]:
idle_data['label'] = 'idle'
running_data['label'] = 'running'
stairs_data['label'] = 'stairs'
walking_data['label'] = 'walking'

combined_data = pd.concat([idle_data, running_data, stairs_data, walking_data], ignore_index=True)

combined_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,label
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking
193856,6.373365,-11.741165,-8.226476,walking
193857,3.289633,-9.993398,-0.383072,walking
193858,-2.978387,-3.050213,1.273715,walking


In [6]:
def calculate_features(df):
    features = pd.DataFrame()
    window_size = 50

    features['mean_x'] = df['accelerometer_X'].rolling(window=window_size).mean()
    features['std_x'] = df['accelerometer_X'].rolling(window=window_size).std()
    features['min_x'] = df['accelerometer_X'].rolling(window=window_size).min()
    features['max_x'] = df['accelerometer_X'].rolling(window=window_size).max()

    features['mean_x'] = df['accelerometer_Y'].rolling(window=window_size).mean()
    features['std_x'] = df['accelerometer_Y'].rolling(window=window_size).std()
    features['min_x'] = df['accelerometer_Y'].rolling(window=window_size).min()
    features['max_x'] = df['accelerometer_Y'].rolling(window=window_size).max()

    features['mean_x'] = df['accelerometer_Z'].rolling(window=window_size).mean()
    features['std_x'] = df['accelerometer_Z'].rolling(window=window_size).std()
    features['min_x'] = df['accelerometer_Z'].rolling(window=window_size).min()
    features['max_x'] = df['accelerometer_Z'].rolling(window=window_size).max()

    return features.dropna()
features = calculate_features(combined_data)

features

,mean_x,std_x,min_x,max_x
49,9.671521,0.323995,8.418014,9.80665
50,9.695559,0.283059,8.418014,9.80665
51,9.721895,0.218362,8.418014,9.80665
52,9.732813,0.205151,8.418014,9.80665
53,9.760107,0.078091,9.251195,9.80665
...,...,...,...,...
193855,0.636092,8.272480,-16.903065,32.59945
193856,0.484586,8.365384,-16.903065,32.59945
193857,0.452217,8.365551,-16.903065,32.59945
193858,0.463422,8.366283,-16.903065,32.59945


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [9]:
labels = combined_data['label'][49:]

X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.3, random_state=42)

In [10]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [11]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [12]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
svm_pred = svm_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

print("SVM :\n", classification_report(y_test, svm_pred))
print("RF :\n", classification_report(y_test, rf_pred))

C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

SVM :
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9291
     running       0.90      0.88      0.89     30609
      stairs       0.00      0.00      0.00      1500
     walking       0.74      0.83      0.78     16744

    accuracy                           0.86     58144
   macro avg       0.66      0.68      0.67     58144
weighted avg       0.84      0.86      0.85     58144

RF :
               precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9291
     running       1.00      1.00      1.00     30609
      stairs       1.00      1.00      1.00      1500
     walking       1.00      1.00      1.00     16744

    accuracy                           1.00     58144
   macro avg       1.00      1.00      1.00     58144
weighted avg       1.00      1.00      1.00     58144

